In [4]:
import numpy as np
import matplotlib.pyplot as plt
import qutip
%matplotlib notebook
from scipy.signal.windows import dpss
from scipy.interpolate import interp1d
import torch

In [5]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    print ("MPS device not found.")

In [6]:
print(torch.__version__)

2.1.0.post100


In [7]:
def make_U(H, time_step, device=device):
    """
    Convert the function to utilize PyTorch and run on a GPU.

    PARAMETERS
        H : Tensor of shape (4,4) representing the time-dependent Hamiltonian
        time_step : float, the time step for integration
        device : string, the device to run the calculations on ('cuda' for GPU or 'cpu' for CPU)

    RETURNS
        U : function of t, which computes the unitary time evolution operator U(t)
    """
    hbar = 1.  # Planck's constant (set to 1 for simplicity)
    I = torch.eye(4, dtype=torch.complex128, device=device)  # Identity matrix

    def U(t):
        total = I  # U(t=0)
        check = [I]
        times = torch.arange(start=0, end=t, step=time_step, device=device)
        for time in times:
            derivative = (-1j / hbar) * torch.matmul(H(time), total) * time_step
            total = total + derivative
            check.append(total) 
        return total, np.array(check)
    return U


In [8]:
def make_H_int(g,delta_omega):
    """
    
    :param g: coupling strength
    :param detuning: omega_q1 - omega_q2
    :return: 
    """
    def H_int(t):
        matrix=np.zeros((4,4),dtype=torch.complex128)
        matrix[2][1] = np.exp((1j)*delta_omega*t)
        matrix[1][2] = np.exp((-1j)*delta_omega*t)
        return g(t)*matrix
    return H_int

In [9]:
def make_H_d1(Omega,V_0,delta_omega,phi,s):
    """
    :param Omega:  
    :param V_0: 
    :param delta_omega:omega_q - omega_d 
    :param phi: offset in driving sine wave
    :param s: control pulse
    :return: 
    """
    I=np.identity(2,dtype=complex)

    def H(t):
        matrix=torch.tensor((2,2),device=device,dtype=torch.complex64)
        matrix[0][1]=np.exp((1j)*(delta_omega*t+phi))
        matrix[1][0]=np.exp((-1j)*(delta_omega*t+phi))
    
        return (-0.5)*Omega*V_0*s(t)*np.kron(matrix,I)
    
    return H #4by4 matrix

In [10]:
matrix=torch.ones((2,2),device=device,dtype=torch.complex)

TypeError: ones() received an invalid combination of arguments - got (tuple, dtype=builtin_function_or_method, device=torch.device), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [11]:
def make_H_d2(Omega,V_0,delta_omega,phi,s):
    """
    :param Omega:  
    :param V_0: 
    :param delta_omega:omega_q - omega_d 
    :param phi: offset in driving sine wave
    :param s: control pulse
    :return: 
    """
    I=np.identity(2,dtype=complex)
    
    def H(t):
        matrix=np.zeros((2,2),dtype='complex128')
        matrix[0][1]=np.exp((1j)*(delta_omega*t+phi))
        matrix[1][0]=np.exp((-1j)*(delta_omega*t+phi))
    
        return (-0.5)*Omega*V_0*s(t)*np.kron(I,matrix)
    
    return H #4by4 matrix

In [12]:
def g(t):
    return 0.04*2*np.pi 
# 40MHz

In [ ]:
def s(t):
    return 1

In [ ]:
def slepian(t):
    Fs = 10 #sampling rate 1ns에 10번 
    N = 500
    time = np.arange(N+1)/Fs 
    #freq = np.fft.fftfreq(N+1,d=1/Fs)
    NW = 6    # Time-halfbandwidth product
    # Generating the Slepian sequences
    slepian_sequence = dpss(N+1, NW)
    slepian_continuous = interp1d(time, slepian_sequence, kind='cubic')
    return slepian_continuous(t)

In [ ]:
Fs = 10 #sampling rate 1ns에 10번 
N = 500
time = np.arange(N+1)/Fs 
    #freq = np.fft.fftfreq(N+1,d=1/Fs)
NW =  3   # Time-halfbandwidth product
    # Generating the Slepian sequences
slepian_sequence = dpss(N+1, NW)
slepian_sequence2 = dpss(N+1, 6)
fig, ax = plt.subplots()
ax.plot(slepian_sequence)
ax.plot(slepian_sequence2)

In [ ]:
#Setting
omega1 = 0.01 * 2 * np.pi #10MHz
omega2 = 0.01 * 2 * np.pi #10MHz
V1=1.
V2=1. #unit?
delta_omega1 = 0.001 * 2 * np.pi #1MHz #qubit frequency and driving frequency
delta_omega2 = 0.001 * 2 * np.pi #1MHz
delta_q12 = 1 * 2 * np.pi # 1GHz
phi1 = 0.001
phi2 = 0.001

In [ ]:
H_int = make_H_int(g=g,delta_omega=delta_omega2) 
H_d1 = make_H_d1(Omega=omega1, V_0=V1,delta_omega=delta_omega1,s=slepian,phi=phi1)
H_d2 = make_H_d2(Omega=omega2, V_0=V2,delta_omega=delta_omega2,s=slepian,phi=phi2)
def H(t):
    return H_int(t)+H_d1(t)+H_d2(t)

In [ ]:
U = make_U(H, time_step=1e-4)

In [ ]:
Uhigh = make_U(H, time_step=1e-5)

In [ ]:
transition, info = U(50.) #한번 돌리는데 4분30초...GPU로 계산하면 더 빠른가?

In [ ]:
transition2, info2 = Uhigh(50.)

In [ ]:
np.save('1e-4_info.npy',info)

In [ ]:
trajectories=[]
fig, ax = plt.subplots()
ax.plot(np.arange(start=0,stop=50.+1e-4,step=1e-4),[u[1,0] for u in info])
    

In [ ]:
unitary_check=np.matmul(np.matrix.getH(transition),transition)
unitary_check

In [ ]:
#prepare qubit 1 in e 
initial_state = np.array([0,0,1.,0,], dtype=complex)
desired_state = np.array([0,1.,0,0,], dtype=complex)

iSWAP=np.zeros((4,4),dtype=complex)
iSWAP[0,0]=1.
iSWAP[3,3]=1.
iSWAP[2,1]=-1j
iSWAP[1,2]=-1j


In [ ]:
trans_state=np.matmul(iSWAP,initial_state)
np.matmul(np.matrix.getH(desired_state),trans_state)

$$s_1(t),\ s_2(t), g(t)$$



In [ ]:
np.matrix.getH(desired_state)

In [ ]:
trans_state